In [1]:
import os
import numpy as np
import pandas as pd
import pyranges as pr
from utils.ref import get_tss_pyranges

In [2]:
pth = os.path.join('..', '..', '..', 'data')
pth_in = os.path.join(pth, 'unzip')
pth_ref = os.path.join(pth, 'ref')
pth_mod = os.path.join(pth, 'features', 'biomart', 'modules')
pth_out = os.path.join(pth, 'DELAY', 'GSE229103')

In [3]:
g = pd.read_csv(os.path.join(pth_mod, 'union.csv')).mmusculus
tss = get_tss_pyranges(os.path.join(pth_ref, 'mm9.refGene.gtf.gz'))

/home/caleb/MesenCoder/DELAY/preprocessing/ground-truth/utils/ref.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_upstream_tss))


In [4]:
g1, g2 = [], []
for src in ('Cdx2', 'T'):
    fn = os.path.join(pth_in, 'GSE84899', f'{src}.txt')
    print(src, fn)
    peaks = pr.read_bed(fn)
    tss_peaks = peaks.nearest(tss, apply_strand_suffix = True)
    g_peaks = tss_peaks.df.gene_id.unique()
    tgt = g_peaks[np.isin(g_peaks, g)].tolist()
    g1.extend([src] * len(tgt)); g2.extend(tgt)

for src in ('Sox2', 'T'):
    fn = os.path.join(pth_in, 'GSE93524', f'{src}.txt')
    print(src, fn)
    g_peaks = pd.read_csv(fn, header = None)[0].unique()
    tgt = g_peaks[np.isin(g_peaks, g)].tolist()
    g1.extend([src] * len(tgt)); g2.extend(tgt)

# write output
refNet = (pd.DataFrame({'Gene1' : g1, 'Gene2' : g2})
          .drop_duplicates()
          .sort_values(['Gene1', 'Gene2']))
fn = os.path.join(pth_out, 'refNetwork.csv')
refNet.to_csv(fn, index = False); refNet

Cdx2 ../../../data/unzip/GSE84899/Cdx2.txt
T ../../../data/unzip/GSE84899/T.txt
Sox2 ../../../data/unzip/GSE93524/Sox2.txt
T ../../../data/unzip/GSE93524/T.txt


,Gene1,Gene2
62,Cdx2,1700017B05Rik
64,Cdx2,Aldh1a2
28,Cdx2,Alpl
16,Cdx2,Alx4
67,Cdx2,Arg1
...,...,...
424,T,Vegfc
425,T,Vim
452,T,Wnt3a
453,T,Zic2


In [5]:
n_edges = refNet.groupby('Gene1').size()
density = (n_edges / g.shape[0])
density.sort_values(ascending = False)

Gene1
T       0.337526
Sox2    0.293501
Cdx2    0.270440
dtype: float64